In [ ]:
import pyqrcode
from PIL import Image
import numpy as np
import os
from random import random, choice, randint
import cv2
from random import *
import os, sys
import string

In [ ]:
IMG_SIZE = 224
LOGO_SIZE = 30
BORDURE=12
Train_DIR='path to train dir'
Test_DIR='path to test dir'

### Generation of images from QR Codes to which Facebook or Twitter logos are added, whole or fragmented, randomly positioned or not.

In [ ]:
def string_generator(length):
    """
    Function that generates a random string of characters of fixed length
    Args :
        lenght (int) : length of the string
        
    Returns :
        string
    """
    str = string.ascii_lowercase
    return ''.join(choice(str) for i in range(length))

In [ ]:
def data_aug(path,num,label):
    """
    Function that performs data augmentation on an image,
    i.e generate 3 images which are rotations (90°,180°,270°) of the original image.
    Args :
        path (string) : path to folder
        num (string) : number of original image
        label (string) : 1 -> facebook 
                          0 -> twitter
    
    """
    img = cv2.imread(path+num+'_'+label+'.png')
    
    img_1 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    cv2.imwrite(path+num+'_1_'+label+'.png', img_1)

    img_2 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    cv2.imwrite(path+num+'_2_'+label+'.png', img_2)


    img_3 = cv2.rotate(img, cv2.ROTATE_180)
    cv2.imwrite(path+num+'_3_'+label+'.png', img_3)

In [ ]:
def put_logo(path,num,logo,position,full):
    """
    function that generates a Qr code with a facebook or twitter logo, 
    whole or fragmented, randomly positioned or not. 
    The call to this function generates and saves an image. 
    
    Args :
        path (string) : path to folder
        num (string) : number of original image
        logo (string) : "fb" -> facebook
                        "twitter" -> twitter
        position (int) : Determines if the logo position is fixed or randomly selected.
                         1 -> fixed
                         0 -> random 
        full (int) : Determines if the logo is whole or fragmented.
                        1 -> whole
                        0 -> fragmented
    """
    #Generation of a QR Code from a string of 150 randomly picked characters
    url = pyqrcode.QRCode(string_generator(150))
    url.png(path+str(num)+'.png',scale=10)
    img = Image.open(path+str(num)+'.png')
    img = img.convert("RGBA")
    img=img.resize((IMG_SIZE,IMG_SIZE))
    
    if full:
        logo = Image.open(logo+'.png')
        logo=logo.resize((LOGO_SIZE,LOGO_SIZE))

        if position :
            xmin = ymin = int((IMG_SIZE / 2) - (LOGO_SIZE / 2))
            xmax = ymax = int((IMG_SIZE / 2) + (LOGO_SIZE / 2))
        else  :
            
            x_min=randint(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE))
            y_min=randint(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE))
    
            x_max=x_min+LOGO_SIZE
            y_max=y_min+LOGO_SIZE
            
        img.paste(logo, (xmin, ymin, xmax, ymax))
        if logo=="fb":
            img.save(path+str(num)+'_1.png') 
        else : 
            img.save(path+str(num)+'_0.png') 
        os.remove(path+str(num)+'.png')
    
    else:
        logo1 = Image.open(logo+'_01.png')
        logo1=logo1.resize((int(LOGO_SIZE/2),LOGO_SIZE))

        logo2 = Image.open(logo+'_02.png')
        logo2=logo2.resize((int(LOGO_SIZE/2),LOGO_SIZE))
            
        x_1 = randrange(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE/2))
        x_2 = randrange(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE/2))

        y_1 = randrange(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE))
        y_2 = randrange(BORDURE,IMG_SIZE-(BORDURE+LOGO_SIZE))
        
        img.paste(logo1, (x_1,y_1))
        img.paste(logo2, (x_2,y_2))
    
        if logo=="fb":
            img.save(path+str(num)+'_1.png') 
        else : 
            img.save(path+str(num)+'_0.png') 
        os.remove(path+str(num)+'.png')

In [ ]:
index=[(i,j) for i in range(224) for j in range(224)] #List of positions (tuple) of each pixel 

In [ ]:
"""
Function that generates training and test images by noising a percentage of the image area.

Args : 
    Train_DIR (string) : path to the training image folder
    
    Test_DIR (string) : path to the test image folder
    
    position (int) : Determines if the logo position is fixed or randomly selected.
                     1 -> fixed
                     0 -> random 
                     
    full (int) : Determines if the logo is whole or fragmented.
                     1 -> whole
                     0 -> fragmented
"""
def genere_img(Train_DIR,Test_DIR,position,full):
    for pourcent in range(0,100,10):
        
        os.mkdir(Train_DIR+'_'+str(pourcent))
        path=Train_DIR+'_'+str(pourcent)+'/'
        bruit=int(IMG_SIZE**2*pourcent/100)
        for i in range(300): #300 images are generated for the training dataset (before data augmentation)
            new=sample(index,bruit)
            put_logo(path,str(i),"fb",position,full)
            o=Image.open(path+str(i)+'_1.png')
            o=o.convert("RGB")
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_1.png')
            data_aug(path,str(i),str(1))
        
        
            put_logo(path,str(i),"twitter",position,full)
            o=Image.open(path+str(i)+'_0.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_0.png')
            data_aug(path,str(i),str(0))
            
    
        os.mkdir(Test_DIR+'_'+str(pourcent))
        path=Test_DIR+'_'+str(pourcent)+'/'    
        for i in range(150): #150 images are generated for the test dataset
            put_logo(path,str(i),"fb",position,full)
            o=Image.open(path+str(i)+'_1.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_1.png')
        
            put_logo(path,str(i),"twitter",position,full)
            o=Image.open(path+str(i)+'_0.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_0.png')

### Generation of images from QR codes to which are added the Facebook or Twitter logos, fragmented into two segments.  The segments are randomly positioned so that a certain distance (from Manhattan) separates them.

In [ ]:
ind=[(i,j) for i in range(20,198) for  j in range(27,183)]

In [ ]:
"""
function that generates a QR code with a Facebook logo fragmented into 2 segments,
randomly positioned at a distance defined by the threshold variable
    
Args :
    path (string) : path to folder
    num (string) : number of original image
    threshold (int) : threshold > 0 
                      The threshold is the Manhattan distance between the barycenters of the two logo fragments
    
    """
    
    
def img_fb(path,num,threshold):
     
    url = pyqrcode.QRCode(genere_phrase(150))
    url.png(path+num+'.png',scale=10)
    img = Image.open(path+num+'.png')
    img = img.convert("RGBA")
    
    img=img.resize((IMG_SIZE,IMG_SIZE))

    logo1 = Image.open('fb_01.png')
    logo1=logo1.resize((16,30))

    logo2 = Image.open('fb_02.png')
    logo2=logo2.resize((16,30))
    

    x_1_min = randint(12,196)
    x_1_max = x_1_min + 16
    
    y_1_min = randint(12,182)
    y_1_max = y_1_min+30
    
    img.paste(logo1, (x_1_min,y_1_min,x_1_max,y_1_max))
    
    bary = ((x_1_min+x_1_max)/2, (y_1_min+y_1_max)/2)
    try :
        alea =sample([i for i in ind if distance.cityblock(bary,i)==threshold],1)
        x_2_min=alea[0][0]-8
        x_2_max=alea[0][0]+8
        y_2_min=alea[0][1]-15
        y_2_max=alea[0][1]+15
        
        img.paste(logo2, (x_2_min,y_2_min,x_2_max,y_2_max))
        img.save(path+num+'_1.png')   
        os.remove(path+num+'.png')
    except ValueError :
        img_fb(path,num,threshold)

In [ ]:
"""
function that generates a QR code with a Twitter logo fragmented into 2 segments,
randomly positioned at a distance defined by the threshold variable
    
Args :
    path (string) : path to folder
    num (string) : number of original image
    threshold (int) : threshold > 0 
                      The threshold is the Manhattan distance between the barycenters of the two logo fragments

"""

def img_twitter(path,num,threshold):
    url = pyqrcode.QRCode(genere_phrase(150))
    url.png(path+num+'.png',scale=10)
    img = Image.open(path+num+'.png')
    img = img.convert("RGBA")
    
    img=img.resize((IMG_SIZE,IMG_SIZE))
    logo1 = Image.open('twitter_01.png')
    logo1=logo1.resize((16,30))

    logo2 = Image.open('twitter_02.png')
    logo2=logo2.resize((16,30))

    x_1_min = randint(12,196)
    x_1_max = x_1_min + 16
    
    y_1_min = randint(12,182)
    y_1_max = y_1_min+30
    
    img.paste(logo1, (x_1_min,y_1_min,x_1_max,y_1_max))
    
    bary = ((x_1_min+x_1_max)/2, (y_1_min+y_1_max)/2)
    try :
        alea =sample([i for i in ind if distance.cityblock(bary,i)==threshold],1)
    
        x_2_min=alea[0][0]-8
        x_2_max=alea[0][0]+8
        y_2_min=alea[0][1]-15
        y_2_max=alea[0][1]+15
        
        img.paste(logo2, (x_2_min,y_2_min,x_2_max,y_2_max))
        img.save(path+num+'_0.png')   
        os.remove(path+num+'.png')
    except ValueError : 
        img_twitter(path,num,threshold)

In [ ]:
#Generating images and adding noise
for threshold in range(50,200,10):
    os.mkdir(Train_DIR+str(threshold))
    for pourcent in range(0,110,10):
        os.mkdir(Train_DIR+str(threshold)+'/train_'+str(pourcent))
        path=Train_DIR+str(threshold)+'/train_'+str(pourcent)+'/'
        print(path)
        bruit=int(IMG_SIZE**2*pourcent/100)
        for i in range(300):
            new=sample(index,bruit)
            img_fb(path,str(i),threshold)
            o=Image.open(path+str(i)+'_1.png')
            o=o.convert("RGB")
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_1.png')
            data_aug(path,str(i),str(1))
        
        
            img_twitter(path,str(i),threshold)
            o=Image.open(path+str(i)+'_0.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_0.png')
            data_aug(path,str(i),str(0))
    
        os.mkdir(Test_DIR+str(threshold)+'/test_'+str(pourcent))
        path=Test_DIR+str(threshold)+'/test_'+str(pourcent)+'/'
        print(path)    
        for i in range(150):
            img_fb(path,str(i),threshold)
            o=Image.open(path+str(i)+'_1.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_1.png')
        
            img_twitter(path,str(i),threshold)
            o=Image.open(path+str(i)+'_0.png')
            o=o.convert("RGB")
            new=sample(index,bruit)
            for n in new:
                o.putpixel(n,(randint(0,255),randint(0,255),randint(0,255)))
            o.save(path+str(i)+'_0.png')